In [ ]:
# import

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import math


In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Hyperparameters
num_epochs = 25
batch_size = 64
learning_rate = 0.001

In [ ]:
# Load CIFAR-100 dataset and apply transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class GaussianNoiseLayer(nn.Module):
    def __init__(self, std_dev):
        super(GaussianNoiseLayer, self).__init__()
        self.std_dev = std_dev
    def forward(self, x):
        noise = torch.randn_like(x) * self.std_dev
        return x + noise

In [ ]:
class LatentSpaceClassfier(nn.Module):
    def __init__(self):
        super(LatentSpaceClassfier, self).__init__()
        self.gaussian_noise = GaussianNoiseLayer(std_dev=0.0)
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.batch1 = nn.BatchNorm2d(16)
        self.batch2 = nn.BatchNorm2d(32)
        self.dropout = nn.Dropout(p=0.5)
        self.softmax = nn.Softmax(dim=1)
        self.fc1 = nn.Linear(32 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 10)
        # self.fc3 = nn.Linear(128, 10)
        self.flatten = nn.Flatten()

        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 8, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = (8*8*16)*x/torch.norm(x)
        x = self.gaussian_noise(x)
        x = self.conv1(x)
        x = self.relu(x)
        # x = self.batch1(x)
        x = self.conv3(x)
        x = self.relu(x)
        # x = self.conv3(x)
        # x = self.relu(x)
        # x = self.batch1(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.conv4(x)
        x = self.relu(x)

        # x = self.conv5(x)
        # x = self.relu(x)

        # x = self.batch2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        # x = self.fc2(x)
        # x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        # x = self.softmax(x)
        return x


In [ ]:
# Set the number of classes in CIFAR-10
num_classes = 10

# Create an instance of the CNN model
model = LatentSpaceClassfier().to(device)

In [ ]:
# define transforms for images
transform = transforms.Compose(
              [transforms.ToTensor(),
              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
cifar10_train = CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar10_test = CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=32, shuffle=True)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Set the model to evaluation mode
model.train()

# Print the modified model architecture
print(model)

LatentSpaceClassfier(
  (gaussian_noise): GaussianNoiseLayer()
  (relu): ReLU()
  (conv1): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (softmax): Softmax(dim=1)
  (fc1): Linear(in_features=2048, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (encoder): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReL

In [ ]:

from torchsummary import summary

summary(model, (3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 16, 16]             448
              ReLU-2           [-1, 16, 16, 16]               0
            Conv2d-3              [-1, 8, 8, 8]           1,160
              ReLU-4              [-1, 8, 8, 8]               0
GaussianNoiseLayer-5              [-1, 8, 8, 8]               0
            Conv2d-6             [-1, 16, 8, 8]           1,168
              ReLU-7             [-1, 16, 8, 8]               0
            Conv2d-8             [-1, 16, 8, 8]           2,320
              ReLU-9             [-1, 16, 8, 8]               0
           Conv2d-10             [-1, 32, 8, 8]           4,640
             ReLU-11             [-1, 32, 8, 8]               0
           Conv2d-12             [-1, 32, 8, 8]           9,248
             ReLU-13             [-1, 32, 8, 8]               0
          Flatten-14                 [-

In [ ]:
# Define the optimizer and the initial learning rate
optimizer = optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

# Define the loss function
criterion = nn.CrossEntropyLoss()

In [ ]:
# Train the model
num_epochs = 40
# Lists to store loss values for plotting
losses = []

for epoch in range(num_epochs):
    # Adjust the learning rate
    model.train()

    scheduler.step()
    total = 0
    correct = 0
    total_loss = 0.0
    # Train the model for one epoch
    for images, labels in train_loader:
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        loss = criterion(outputs, labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    accuracy = correct / total
    # Compute average loss for the epoch
    avg_loss = total_loss / len(train_loader)
    losses.append(avg_loss)
    # Print the loss after each epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()},Accuracy:{100 * accuracy:.2f}%")

    # Change learning rate after 25 epochs
    if epoch == 30:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.0001

Epoch [1/40], Loss: 1.4986178874969482,Accuracy:38.79%
Epoch [2/40], Loss: 0.8187788128852844,Accuracy:49.76%
Epoch [3/40], Loss: 0.8114129304885864,Accuracy:54.20%
Epoch [4/40], Loss: 1.4382719993591309,Accuracy:57.70%
Epoch [5/40], Loss: 0.899817943572998,Accuracy:60.20%
Epoch [6/40], Loss: 0.9540510177612305,Accuracy:62.09%
Epoch [7/40], Loss: 0.9085034728050232,Accuracy:63.63%
Epoch [8/40], Loss: 1.274178385734558,Accuracy:65.35%
Epoch [9/40], Loss: 1.573726773262024,Accuracy:66.46%
Epoch [10/40], Loss: 1.3145601749420166,Accuracy:67.54%
Epoch [11/40], Loss: 1.1271247863769531,Accuracy:68.31%
Epoch [12/40], Loss: 1.2179853916168213,Accuracy:69.24%
Epoch [13/40], Loss: 0.49981650710105896,Accuracy:70.26%
Epoch [14/40], Loss: 1.3311123847961426,Accuracy:71.14%
Epoch [15/40], Loss: 0.9373960494995117,Accuracy:71.57%
Epoch [16/40], Loss: 0.3857816755771637,Accuracy:73.04%
Epoch [17/40], Loss: 0.3188841938972473,Accuracy:73.43%
Epoch [18/40], Loss: 0.5792578458786011,Accuracy:74.10%
Epo

In [ ]:

model.eval()  # Set the model to evaluation mode

total = 0
correct = 0

with torch.no_grad():
    for images, labels in test_loader:
        # Forward pass
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)

        # Get the predicted labels
        _, predicted = torch.max(outputs.data, 1)

        # Update total and correct predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate the accuracy
accuracy = correct / total

# Print the accuracy on the test set
print(f"Accuracy on the test set: {100 * accuracy:.2f}%")


Accuracy on the test set: 63.75%
